In [8]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import sys
import importlib

# Determine root directory - go up from notebooks folder to portfolio_analyser
if Path.cwd().name == 'notebooks':
    ROOT = Path.cwd().parent
else:
    ROOT = Path.cwd()

src_path = ROOT / 'src'

# Add src to path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Reload modules to pick up any code changes without restarting kernel
if 'technical_analysis' in sys.modules:
    import technical_analysis
    importlib.reload(technical_analysis)
    print("[OK] Reloaded technical_analysis module")

if 'full_scanner' in sys.modules:
    import full_scanner
    importlib.reload(full_scanner)
    print("[OK] Reloaded full_scanner module")

from full_scanner import (
    get_sp500_tickers,
    get_nasdaq100_tickers,
    get_portfolio_tickers,
    scan_stocks,
    filter_buy_signals,
    filter_sell_signals,
    create_excel_output,
    create_portfolio_excel,
    create_pdf_report,
    create_best_trades_excel,
    create_best_trades_pdf,
    cleanup_old_scans
)

# Setup results directory
results_dir = ROOT / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print("[OK] Scanner modules loaded")
print(f"[OK] Results directory: {results_dir}")
print(f"[OK] Timestamp: {timestamp}")

[OK] Reloaded technical_analysis module
[OK] Reloaded full_scanner module
[OK] Scanner modules loaded
[OK] Results directory: c:\workspace\portfolio_analyser\scanner_results
[OK] Timestamp: 20260112_2020


## Configuration

Set your scan parameters:
- **daily_bars**: Number of daily bars to analyze (default: 60)
- **weekly_bars**: Number of weekly bars to analyze (default: 52)
- **concurrency**: Number of parallel workers (default: 2, max recommended: 5)

In [9]:
# Scan parameters
DAILY_BARS = 60
WEEKLY_BARS = 52
CONCURRENCY = 2  # Be careful with higher values (rate limits)

print(f"Daily bars: {DAILY_BARS}")
print(f"Weekly bars: {WEEKLY_BARS}")
print(f"Concurrency: {CONCURRENCY}")

Daily bars: 60
Weekly bars: 52
Concurrency: 2


## Market Regime Analysis

Analyze the current market regime using QQQ (NASDAQ 100 ETF) and SPY (S&P 500 ETF). This determines whether we should be aggressive, cautious, or defensive with new trades.

In [10]:
from full_scanner import analyze_market_regime

# Analyze NASDAQ 100 regime using QQQ
print("=" * 80)
print("ANALYZING MARKET REGIMES")
print("=" * 80)

qqq_regime = analyze_market_regime(daily_bars=DAILY_BARS, weekly_bars=WEEKLY_BARS, ticker="QQQ")

# Display QQQ regime information
print(f"\n{'='*80}")
print(f"QQQ (NASDAQ 100) REGIME: {qqq_regime['tier']}")
print(f"{'='*80}")
print(f"Signal: {qqq_regime['signal']}")
print(f"Weekly: {qqq_regime['weekly_state']} | Daily: {qqq_regime['daily_state']}")
if qqq_regime.get('etf_price') and qqq_regime.get('etf_rsi'):
    print(f"Price: ${qqq_regime['etf_price']:.2f} | RSI: {qqq_regime['etf_rsi']:.1f}")
print(f"\nTrade Restrictions: Allow Buys = {qqq_regime['allow_buys']}")
print(f"{'='*80}\n")

# Analyze S&P 500 regime using SPY
spy_regime = analyze_market_regime(daily_bars=DAILY_BARS, weekly_bars=WEEKLY_BARS, ticker="SPY")

# Display SPY regime information
print(f"\n{'='*80}")
print(f"SPY (S&P 500) REGIME: {spy_regime['tier']}")
print(f"{'='*80}")
print(f"Signal: {spy_regime['signal']}")
print(f"Weekly: {spy_regime['weekly_state']} | Daily: {spy_regime['daily_state']}")
if spy_regime.get('etf_price') and spy_regime.get('etf_rsi'):
    print(f"Price: ${spy_regime['etf_price']:.2f} | RSI: {spy_regime['etf_rsi']:.1f}")
print(f"\nTrade Restrictions: Allow Buys = {spy_regime['allow_buys']}")
print(f"{'='*80}\n")

ANALYZING MARKET REGIMES

MARKET REGIME ANALYSIS (QQQ - NASDAQ 100 ETF)

🟢 MARKET REGIME: GREEN
   QQQ Signal: FULL HOLD + ADD
   Weekly State: N/A | Daily State: N/A
   QQQ Price: $627.46 | RSI: 64.0

   FULL BULL MARKET - Trade normally


QQQ (NASDAQ 100) REGIME: GREEN
Signal: FULL HOLD + ADD
Weekly: N/A | Daily: N/A
Price: $627.46 | RSI: 64.0

Trade Restrictions: Allow Buys = True


MARKET REGIME ANALYSIS (SPY - S&P 500 ETF)

🟢 MARKET REGIME: GREEN
   SPY Signal: FULL HOLD + ADD
   Weekly State: N/A | Daily State: N/A
   SPY Price: $694.96 | RSI: 70.0

   FULL BULL MARKET - Trade normally


SPY (S&P 500) REGIME: GREEN
Signal: FULL HOLD + ADD
Weekly: N/A | Daily: N/A
Price: $694.96 | RSI: 70.0

Trade Restrictions: Allow Buys = True



## Scan Portfolio Stocks

Quick scan of your portfolio stocks from stocks.txt. Shows both buy opportunities (FULL HOLD + ADD) and sell signals (bearish signals).

In [11]:
print("Loading portfolio tickers from stocks.txt...")
portfolio_tickers = get_portfolio_tickers()

if portfolio_tickers:
    print(f"Found {len(portfolio_tickers)} portfolio stocks\n")

    # Run scan - Portfolio uses QQQ regime (tech-heavy portfolio)
    portfolio_results = scan_stocks(
        portfolio_tickers,
        category="Portfolio",
        daily_bars=DAILY_BARS,
        weekly_bars=WEEKLY_BARS,
        concurrency=CONCURRENCY
    )

    if not portfolio_results.empty:
        # Get buy signals (all and quality-filtered) - Uses QQQ regime
        portfolio_buy_all = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD', quality_filter=False, regime=qqq_regime)
        portfolio_buy = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD', quality_filter=True, regime=qqq_regime)
        
        # Get sell signals (all bearish signals)
        portfolio_sell = filter_sell_signals(portfolio_results, quality_filter=False)
        
        # Display summary
        print(f"\n[STATS] Portfolio Results (QQQ Regime: {qqq_regime['tier']}):")
        print(f"   [OK] {len(portfolio_buy)} buy signals (EXCELLENT/GOOD/OK)")
        if len(portfolio_buy_all) > len(portfolio_buy):
            print(f"   [WAIT] {len(portfolio_buy_all) - len(portfolio_buy)} extended/weak (wait)")
        print(f"   [SELL] {len(portfolio_sell)} sell signals (bearish)")
        
        # Display buy opportunities
        if not portfolio_buy.empty:
            print("\n[OK] BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
            display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
            if all(col in portfolio_buy.columns for col in display_cols):
                print(portfolio_buy[display_cols].to_string(index=False))
            else:
                print(portfolio_buy[['ticker', 'signal', 'current_price']].to_string(index=False))
        
        # Display sell signals
        if not portfolio_sell.empty:
            print("\n[SELL] SELL SIGNALS (Bearish - reduce exposure):")
            display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
            if all(col in portfolio_sell.columns for col in display_cols):
                print(portfolio_sell[display_cols].to_string(index=False))
            else:
                print(portfolio_sell[['ticker', 'signal', 'current_price']].to_string(index=False))
        
        # Save combined results
        if not portfolio_buy.empty or not portfolio_sell.empty:
            # Create portfolio subfolder
            portfolio_dir = results_dir / 'portfolio'
            portfolio_dir.mkdir(exist_ok=True)
            
            xlsx_path = portfolio_dir / f'portfolio_playbook_{timestamp}.xlsx'
            pdf_path = portfolio_dir / f'portfolio_playbook_{timestamp}.pdf'
            best_trades_xlsx = portfolio_dir / f'portfolio_watchlist_{timestamp}.xlsx'
            best_trades_pdf = portfolio_dir / f'portfolio_watchlist_{timestamp}.pdf'
            
            # Create standard reports with both buy and sell data
            create_excel_output(portfolio_buy, portfolio_sell, xlsx_path, category="Portfolio")
            create_pdf_report(portfolio_buy, portfolio_sell, pdf_path, timestamp, category="Portfolio", regime=qqq_regime)
            
            # Create best trades reports
            create_best_trades_excel(portfolio_buy, portfolio_sell, best_trades_xlsx, category="Portfolio")
            create_best_trades_pdf(portfolio_buy, portfolio_sell, best_trades_pdf, category="Portfolio", regime=qqq_regime)
            
            print(f"\n[OK] Excel: portfolio/{xlsx_path.name}")
            print(f"[OK] PDF: portfolio/{pdf_path.name}")
            print(f"[OK] Best Trades Excel: portfolio/{best_trades_xlsx.name}")
            print(f"[OK] Best Trades PDF: portfolio/{best_trades_pdf.name}")
else:
    print("[WARN] No portfolio tickers found in stocks.txt")

Loading portfolio tickers from stocks.txt...
[OK] Loaded 9 portfolio tickers from stocks.txt

Found 9 portfolio stocks

[SCAN] Scanning 9 Portfolio stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [1/9] TSLA   -> FULL HOLD + ADD      $450.09 | Entry: CAUTION    ⚠ THIN
[OK] [2/9] ASML   -> FULL HOLD + ADD      $1,277.31 | Entry: CAUTION    ⚠ EXTENDED
[OK] [3/9] NVDA   -> FULL HOLD + ADD      $186.71 | Entry: CAUTION    ⚠ THIN
[SELL] [5/9] ALAB   -> HOLD MOST + REDUCE   $174.00 | Entry: CAUTION    ⚠ EXTENDED
[OK] [7/9] MRVL   -> FULL HOLD + ADD      $83.12 | Entry: GOOD       ✓ SAFE ENTRY
[SELL] [8/9] BTC-USD -> HOLD MOST + REDUCE   $91,381.09 | Entry: GOOD       ✓ IDEAL
[SELL] [9/9] SOL-USD -> CASH                 $140.99 | Entry: GOOD       ✓ IDEAL

[OK] Scan complete: 9 analyzed, 4 FULL HOLD + ADD signals found


[STATS] Portfolio Results (QQQ Regime: GREEN):
   [OK] 1 buy signals (EXCELLENT/GOOD/OK)
   [WAIT] 3 extended/weak (wait)
   

## Scan S&P 500

Full scan of ~500 stocks. Takes 5-10 minutes depending on cache. Shows both buy opportunities (FULL HOLD + ADD) and sell signals (bearish signals).

In [12]:
print("Fetching S&P 500 ticker list...")
sp500_tickers = get_sp500_tickers()
print(f"Found {len(sp500_tickers)} S&P 500 stocks\n")

# Run scan - S&P 500 uses SPY regime
sp500_results = scan_stocks(
    sp500_tickers,
    category="S&P 500",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not sp500_results.empty:
    # Get buy signals (all and quality-filtered) - Uses SPY regime
    sp500_buy_all = filter_buy_signals(sp500_results, 'FULL HOLD + ADD', quality_filter=False, regime=spy_regime)
    sp500_buy = filter_buy_signals(sp500_results, 'FULL HOLD + ADD', quality_filter=True, regime=spy_regime)
    
    # Get sell signals (all bearish signals, matching portfolio_analysis logic)
    sp500_sell = filter_sell_signals(sp500_results, quality_filter=False)
    
    # Display summary
    print(f"\n[STATS] S&P 500 Results (SPY Regime: {spy_regime['tier']}):")
    print(f"   [OK] {len(sp500_buy)} buy signals (EXCELLENT/GOOD/OK)")
    if len(sp500_buy_all) > len(sp500_buy):
        print(f"   [WAIT] {len(sp500_buy_all) - len(sp500_buy)} extended/weak (wait)")
    print(f"   [SELL] {len(sp500_sell)} sell signals (bearish)")
    
    # Display buy opportunities
    if not sp500_buy.empty:
        print("\n[OK] TOP BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
        display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
        if all(col in sp500_buy.columns for col in display_cols):
            print(sp500_buy[display_cols].head(10).to_string(index=False))
        else:
            print(sp500_buy[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Display sell signals (matches portfolio_analysis behavior)
    if not sp500_sell.empty:
        print("\n[SELL] SELL SIGNALS (Bearish - reduce exposure):")
        display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
        if all(col in sp500_sell.columns for col in display_cols):
            print(sp500_sell[display_cols].head(10).to_string(index=False))
        else:
            print(sp500_sell[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Save combined results if we have either buy or sell signals
    if not sp500_buy.empty or not sp500_sell.empty:
        # Create sp500 subfolder
        sp500_dir = results_dir / 'sp500'
        sp500_dir.mkdir(exist_ok=True)
        
        xlsx_path = sp500_dir / f'sp500_playbook_{timestamp}.xlsx'
        pdf_path = sp500_dir / f'sp500_playbook_{timestamp}.pdf'
        best_trades_xlsx = sp500_dir / f'sp500_watchlist_{timestamp}.xlsx'
        best_trades_pdf = sp500_dir / f'sp500_watchlist_{timestamp}.pdf'
        
        # Create standard reports with both buy and sell data
        create_excel_output(sp500_buy, sp500_sell, xlsx_path, category="S&P 500")
        create_pdf_report(sp500_buy, sp500_sell, pdf_path, timestamp, category="S&P 500", regime=spy_regime)
        
        # Create best trades reports
        create_best_trades_excel(sp500_buy, sp500_sell, best_trades_xlsx, category="S&P 500")
        create_best_trades_pdf(sp500_buy, sp500_sell, best_trades_pdf, category="S&P 500", regime=spy_regime)
        
        print(f"\n[OK] Excel: sp500/{xlsx_path.name}")
        print(f"[OK] PDF: sp500/{pdf_path.name}")
        print(f"[OK] Best Trades Excel: sp500/{best_trades_xlsx.name}")
        print(f"[OK] Best Trades PDF: sp500/{best_trades_pdf.name}")
else:
    print("[WARN] No results from S&P 500 scan")

Fetching S&P 500 ticker list...
[OK] Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

[SCAN] Scanning 503 S&P 500 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [1/503] MMM    -> FULL HOLD + ADD      $167.12 | Entry: CAUTION    ⚠ THIN
[SELL] [2/503] AOS    -> CASH                 $71.11 | Entry: GOOD       ✓ IDEAL
[SELL] [3/503] ABT    -> HOLD MOST + REDUCE   $124.10 | Entry: CAUTION    ⚠ EXTENDED
[SELL] [4/503] ACN    -> REDUCE               $279.93 | Entry: OK         ✓ ACCEPTABLE
[OK] [5/503] ABBV   -> FULL HOLD + ADD      $219.35 | Entry: OK         ✓ ACCEPTABLE
[SELL] [6/503] ADBE   -> CASH                 $330.27 | Entry: GOOD       ✓ IDEAL
[OK] [9/503] A      -> FULL HOLD + ADD      $147.75 | Entry: OK         ✓ ACCEPTABLE
[SELL] [10/503] APD    -> FULL CASH / DEFEND   $266.46 | Entry: GOOD       ✓ IDEAL
[OK] [11/503] AFL    -> FULL HOLD + ADD      $108.77 | Entry: GOOD       ✓ IDEAL
[SELL] [12/503] ABNB   -> REDUCE        

## Scan NASDAQ 100

Scan ~100 tech-heavy stocks. Takes 2-5 minutes. Shows both buy opportunities (FULL HOLD + ADD) and sell signals (bearish signals).

In [13]:
print("Fetching NASDAQ 100 ticker list...")
nasdaq100_tickers = get_nasdaq100_tickers()
print(f"Found {len(nasdaq100_tickers)} NASDAQ 100 stocks\n")

# Run scan - NASDAQ 100 uses QQQ regime
nasdaq100_results = scan_stocks(
    nasdaq100_tickers,
    category="NASDAQ 100",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not nasdaq100_results.empty:
    # Get buy signals (all and quality-filtered) - Uses QQQ regime
    nasdaq100_buy_all = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD', quality_filter=False, regime=qqq_regime)
    nasdaq100_buy = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD', quality_filter=True, regime=qqq_regime)
    
    # Get sell signals (all bearish signals, matching portfolio_analysis logic)
    nasdaq100_sell = filter_sell_signals(nasdaq100_results, quality_filter=False)
    
    # Display summary
    print(f"\n[STATS] NASDAQ 100 Results (QQQ Regime: {qqq_regime['tier']}):")
    print(f"   [OK] {len(nasdaq100_buy)} buy signals (EXCELLENT/GOOD/OK)")
    if len(nasdaq100_buy_all) > len(nasdaq100_buy):
        print(f"   [WAIT] {len(nasdaq100_buy_all) - len(nasdaq100_buy)} extended/weak (wait)")
    print(f"   [SELL] {len(nasdaq100_sell)} sell signals (bearish)")
    
    # Display buy opportunities
    if not nasdaq100_buy.empty:
        print("\n[OK] TOP BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
        display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
        if all(col in nasdaq100_buy.columns for col in display_cols):
            print(nasdaq100_buy[display_cols].head(10).to_string(index=False))
        else:
            print(nasdaq100_buy[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Display sell signals (matches portfolio_analysis behavior)
    if not nasdaq100_sell.empty:
        print("\n[SELL] SELL SIGNALS (Bearish - reduce exposure):")
        display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
        if all(col in nasdaq100_sell.columns for col in display_cols):
            print(nasdaq100_sell[display_cols].head(10).to_string(index=False))
        else:
            print(nasdaq100_sell[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Save combined results if we have either buy or sell signals
    if not nasdaq100_buy.empty or not nasdaq100_sell.empty:
        # Create nasdaq100 subfolder
        nasdaq100_dir = results_dir / 'nasdaq100'
        nasdaq100_dir.mkdir(exist_ok=True)
        
        xlsx_path = nasdaq100_dir / f'nasdaq100_playbook_{timestamp}.xlsx'
        pdf_path = nasdaq100_dir / f'nasdaq100_playbook_{timestamp}.pdf'
        best_trades_xlsx = nasdaq100_dir / f'nasdaq100_watchlist_{timestamp}.xlsx'
        best_trades_pdf = nasdaq100_dir / f'nasdaq100_watchlist_{timestamp}.pdf'
        
        # Create standard reports with both buy and sell data
        create_excel_output(nasdaq100_buy, nasdaq100_sell, xlsx_path, category="NASDAQ 100")
        create_pdf_report(nasdaq100_buy, nasdaq100_sell, pdf_path, timestamp, category="NASDAQ 100", regime=qqq_regime)
        
        # Create best trades reports
        create_best_trades_excel(nasdaq100_buy, nasdaq100_sell, best_trades_xlsx, category="NASDAQ 100")
        create_best_trades_pdf(nasdaq100_buy, nasdaq100_sell, best_trades_pdf, category="NASDAQ 100", regime=qqq_regime)
        
        print(f"\n[OK] Excel: nasdaq100/{xlsx_path.name}")
        print(f"[OK] PDF: nasdaq100/{pdf_path.name}")
        print(f"[OK] Best Trades Excel: nasdaq100/{best_trades_xlsx.name}")
        print(f"[OK] Best Trades PDF: nasdaq100/{best_trades_pdf.name}")
else:
    print("[WARN] No results from NASDAQ 100 scan")

Fetching NASDAQ 100 ticker list...
[OK] Loaded 101 NASDAQ 100 tickers

Found 101 NASDAQ 100 stocks

[SCAN] Scanning 101 NASDAQ 100 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[SELL] [2/101] ADBE   -> CASH                 $330.27 | Entry: GOOD       ✓ IDEAL
[SELL] [3/101] ABNB   -> REDUCE               $138.22 | Entry: CAUTION    ⚠ EXTENDED
[OK] [4/101] GOOGL  -> FULL HOLD + ADD      $329.92 | Entry: CAUTION    ⚠ EXTENDED
[SELL] [5/101] ALNY   -> HOLD MOST + REDUCE   $365.65 | Entry: GOOD       ✓ IDEAL
[OK] [6/101] GOOG   -> FULL HOLD + ADD      $330.49 | Entry: CAUTION    ⚠ EXTENDED
[OK] [7/101] AMZN   -> FULL HOLD + ADD      $247.59 | Entry: CAUTION    ⚠ THIN
[OK] [9/101] AMGN   -> FULL HOLD + ADD      $324.68 | Entry: EXCELLENT  ✓ SAFE ENTRY
[OK] [10/101] ADI    -> FULL HOLD + ADD      $295.51 | Entry: CAUTION    ⚠ EXTENDED
[OK] [11/101] AAPL   -> FULL HOLD + ADD      $260.42 | Entry: CAUTION    ⚠ THIN
[OK] [12/101] AMAT   -> FULL HOLD

## Cleanup Old Scan Files

Archive old scan results to keep the scanner_results folder clean. Keeps the 2 most recent files per category (Portfolio, S&P 500, NASDAQ 100).

In [14]:
print("\n" + "="*80)
print("CLEANUP - Archiving Old Scan Files")
print("="*80)

# Keep 1 most recent scan per category (4 files per scan: 2 standard + 2 best trades)
# Archives older files, deletes archives older than 60 days
cleanup_old_scans(results_dir, max_files=1, archive_retention_days=60)

print("\n[OK] Cleanup complete")


CLEANUP - Archiving Old Scan Files
  [OK] No files to archive (only 1 most recent scan set(s) exist)

[OK] Cleanup complete
